In [1]:
#################################################################################
######     Plot Eff, Nef and Mov for a Picard number three variety   ############
#################################################################################

In [2]:
from sage.numerical.mip import MixedIntegerLinearProgram
from scipy.spatial import ConvexHull
%run ToricFunctions.ipynb

In [3]:
##############################################
#######  Define the polygon to be used #######
##############################################

def reorder(A):
    pivots=A.pivots()
    new_order = list(pivots) + [i for i in range(A.ncols()) if i not in pivots]
    return A[:, new_order]
    
def polyplot(X):
    B=reorder(matrix(eff_cone(X).rays()).transpose())
    P=[0]*B.ncols()
    P[0]=vector([-1,0])
    P[1]=vector([0,2])
    P[2]=vector([1,0])
    for i in range(3,B.ncols()):
        P[i]=B.rref()[0,i]*P[0]+B.rref()[1,i]*P[1]+B.rref()[2,i]*P[2]
    return P

def plane_coord(X,A):
    M=matrix(polyplot(X))
    MD=matrix(A)
    return vector(MD*M)

In [14]:
####################################################
#######  Generalized barycentric coordinates #######
####################################################

def normalize(A):
    w=0
    B=vector(A)
    for j in [0..len(A)-1]:
        w=w+A[j]
    if w==0:
        return vector(A)    
    return B/w
    
def coeff(D,rays):
    cone = Cone(rays)
    v = list(D)  # Example ray in the interior of the cone

    # Set up the linear program
    lp = MixedIntegerLinearProgram()
    lambdas = lp.new_variable(nonnegative=True)  # Coefficients must be non-negative

    # Add constraints to express v as a positive combination of the rays
    for i in range(len(v)):
        lp.add_constraint(sum(lambdas[j] * rays[j][i] for j in range(len(rays))) == v[i])

    lp.set_objective(lp.sum(lambdas[j] for j in range(len(rays))))
    lp.solve()
    coefficients = [lp.get_values(lambdas[j]) for j in range(len(rays))]
    return normalize(coefficients)

############################
######## Plot cones ########
############################

###### Basis of Pic consisting of classes of invariant divisors

def basis_eff(X):
    be=matrix(eff_cone(X))
    ind_rows=((be.transpose()).echelon_form()).pivots()
    basis=[be[i] for i in ind_rows]
    return basis, (matrix(basis).transpose()).inverse()

###############################
####### Coordinate change  ####
###############################

def change_be(X,D):
    return basis_eff(X)[1]*vector(D)

def change_eb(X,D):
    return (basis_eff(X)[1].inverse())*vector(D) 

#####################################################
###### Change coordinates to coordinates in Eff #####
#####################################################

def change_be_cone(X,C):
    rays=C.rays()
    l=[]
    for i in range(len(rays)):
        l.append(change_be(X,rays[i]))
    return l

###################################
###### Plot a cone in a plane #####
###################################

def plot_cone(L,color='blue'):
    hull = ConvexHull(L)
    points = [L[i] for i in hull.vertices]
    return polygon2d(points, fill=True, color=color, alpha=0.2,thickness=2)

def plot_cone_empty(L,color='blue'):
    hull = ConvexHull(L)
    points = [L[i] for i in hull.vertices]
    return polygon2d(points, fill=False, color=color,alpha=0.4,thickness=1.5)

#######################################
###### Plot relevant cones in Eff #####
#######################################

def list_to_points(X,L):
    L1=[]
    for i in range(len(L)):
        L1.append(plane_coord(X,coeff(L[i],change_be_cone(X,eff_cone(X)))))
    return L1

def effplot(X):
    EE=change_be_cone(X,eff_cone(X))
    ME=change_be_cone(X,mov_cone(X))
    NE=change_be_cone(X,nef_cone(X))
    EP=plot_cone_empty(list_to_points(X,EE),color='blue')
    NP=plot_cone(list_to_points(X,NE),color='red')
    MP=plot_cone(list_to_points(X,ME),color='orange')
    return EP,MP,NP

###############################
######## Plot divisors ########
###############################

######## Plot a divisors (given as a vector) ########

def plotdivisor(X,D,color='blue',name=None):
    c1=plane_coord(X,coeff(change_be(X,Pic(X,D)),change_be_cone(X,eff_cone(X))))
    p=list_plot([c1], color=color,size=30)
    if name is not None:
        t = text(f"{name}", vector(div_in_plane(X,Pic(X,D)))+vector(QQ,[0,0.1]), fontsize=10, color='black')  
        return p+t
    else:  
        return p 
    
######## Plot an ample divisor  ########

def plotample(X):
    a=list(unPic(X,ample_divisor(X)))
    return plotdivisor(X,a,color='green')

######## Plot the anticanonical divisor ########

def plotanticanonical(X):
    k=[1]*len(X.fan().rays())
    return plotdivisor(X,k,color='red')    

######## Plot invariant divisors ########

def invplot(X):
    r=X.fan().rays()
    I = identity_matrix(QQ, len(r))
    rows = [list(row) for row in I.rows()]
    D=[]
    for i in range(len(r)):
        D.append(plotdivisor(X,rows[i],color='blue'))
    return D

def div_in_plane(X,D):
    return plane_coord(X,coeff(change_be(X,D),change_be_cone(X,eff_cone(X))))

def invpoints(X):
    Deff=invariant_in_pic(X)
    Peff=[]
    for i in range(len(Deff)):
        Peff.append(div_in_plane(X,Deff[i]))
    return Peff

######## Plot cones of secondary fan ########

def plotsecfan(X):
    points=[]
    for i in range(len(secondary_fan(X)[0])):
         points.append(div_in_plane(X,(secondary_fan(X)[0])[i]))   
    csf=[[points[i] for i in sublist] for sublist in (secondary_fan(X))[1]]
    plotcsf=[plot_cone_empty(cone) for cone in csf] 
    return plotcsf

######## Show cones #########

def showeff(X):
    return sum(plotsecfan(X))+sum(effplot(X))+sum(invplot(X))

def plotmodel(X,a,name=None):
    if name is not None:
        D=plotdivisor(X,chambers(X)[a],color='red',name=name)
        return show(showeff(X)+D,axes=false)
    else:
        D=plotdivisor(X,chambers(X)[a],color='red')
        return show(showeff(X)+D,axes=false)

def plotmodels(X):
    D=[]
    for i in range(len(chambers(X))):
        D.append(plotdivisor(X,chambers(X)[i],color='red',name=i))
    return show(showeff(X)+sum(D),axes=false)    